Small command to ensure this code runs correctly on all notebooks:

In [ ]:
!pip install numpy pandas scikit-learn

___
## **1 | Search Algorithm Implementation.**

---

## 1.1 | Problem Description.

### 1.1.1 | Initial State.

To initialise the problem, the elements that make up the initial state are defined.

This includes the layout of the bays, all positional indicators, the held container, and a dictionary containing the weights of the boxes.

For this implementation, all containers other than 2 and 4 are treated as light, so specifically defining them is unnecessary.

These elements are then put together into a list, named "initial_state".

The possible actions are also defined here, in a list.

In [ ]:
initial_bays = [[],[],[],[2,3,4],[1,5],[],[],[6]]
initial_agent_pos = 0
initial_held_container = None
initial_cost = 0
initial_target_bay = 0

weight = {2: 'heavy', 4: 'heavy'}

initial_state = [initial_bays, initial_agent_pos, initial_held_container, initial_target_bay, weight]

all_actions = ["LEFT", "RIGHT", "PICK", "DROP"]

### 1.1.2 | Goal State.

To finish building the problem, a goal state is defined. The function "is_goal_state(state)" can be called to determine if a given state meets the conditions required. These being; The bay at the agent's position being empty, and the target bay being the location of containers 1,2 and 3, sequentially.

The goal state function can be tested by calling "is_goal_state(initial_state)", which gives "False", as the initial state does not meet the criteria. Rearranging the initial state to satisfy the goal will give "True".

The final addition to the goal state defines that the held container is none. While this is not strictly in the goal state requirements, it helps avoid an issue where the agent picks up a box, thereby satisfying the goal condition and returning the path, when moving over the obstruction is almost always slightly more efficient in terms of cost.

In [ ]:
def is_goal_state(state):
    if state[0][state[1]] == [] and state[0][state[3]] == [1, 2, 3] and state[2] == None:
        return True

    return False

is_goal_state(initial_state)

___

## 1.2 | Basic Functions.

### 1.2.1 | Print State.

Next, the basic functionality allowing a state to be printed is defined in the function "print_state(state)". This is unnecessary for the solution, but allows both any given state to be printed, and for the solution to be animated later.

This function can be tested by calling "print_state(initial_state)", which gives a graphical representation of the initial state, with all relevant information printed.

In [ ]:
def print_state(state):
    for i in range(len(state[0])):
        if i == state[1]:
            print(str(state[0][i]) + " <-[" + str(state[2]) + "]-")
        else:
            print(str(state[0][i]))
    print("\nTarget bay: " + str(state[3]))
    print()

print_state(initial_state)

### 1.2.2 | Is Action Valid.

The function allowing action validity to be determined is defined as "is_action_valid(state, action, prev_action)".

These arguments allow for a significantly less complicated search tree compared to omitting this function.

Any function that is the direct opposite of the previously made action is invalid, as that would be redundant.

Also outlined here is logic disallowing "PICK" and "DROP" when the held container is "not None" and "None" respectively.

This also disallows "DROP" when the current bay is full, and "PICK" when the current bay is empty.

"LEFT" and "RIGHT" cannot be taken if the agent is at the left boundary and right boundary respectively.

The underscores are used as junk variables, to deconstruct state properly. They are not useful to this specific function.

In [ ]:
def is_action_valid(state, action, prev_action):
    
    bays, agent_pos, held_container, _, _ = state

    if prev_action == "LEFT" and action == "RIGHT" or \
       prev_action == "RIGHT" and action == "LEFT" or \
       prev_action == "PICK" and action == "DROP" or \
       prev_action == "DROP" and action == "PICK":
        return False

    if action == "PICK" and not bays[agent_pos]:
        return False

    if action == "PICK" and held_container is not None:
        return False

    if action == "DROP" and len(bays[agent_pos]) >= 3:
        return False

    if action == "DROP" and held_container is None:
        return False

    if action == "LEFT" and agent_pos == 0:
        return False

    if action == "RIGHT" and agent_pos == 7:
        return False

    return True

### 1.2.3 | Apply Action.

The function to apply a given action to a given state is defined as "apply_action(state, action, prev_action=None)".

For this function to work, copy is imported, as it is needed to create a working duplicate of the state.

First, the action validity is checked. This helps minimize the search tree complexity. If the action is not valid, the original state is returned.

This will be ignored by the searching algorithm, as repeat states are skipped.

Each action is then defined along with its impact on the state, and the associated cost.

The listed cost is the minutes required to perform the action.

It takes into account the weight of the held container for "LEFT" and "RIGHT", and the state of the current bay for "PICK".

In [ ]:
import copy

def apply_action(state, action, prev_action=None):
    
    if not is_action_valid(state, action, prev_action):
        return state, 0

    new_state = copy.deepcopy(state)
    cost = 0

    if action == "LEFT":
        new_state[1] -= 1
        if new_state[4].get(new_state[2]) == "heavy":
            cost = 5
        else:
            cost = 3

    if action == "RIGHT":
        new_state[1] += 1
        if new_state[4].get(new_state[2]) == "heavy":
            cost = 5
        else:
            cost = 3

    if action == "PICK":
        cost = 4 - len(new_state[0][new_state[1]])
        new_state[2] = new_state[0][new_state[1]].pop()

    if action == "DROP":
        new_state[0][new_state[1]].append(new_state[2])
        new_state[2] = None
        cost = 2

    return new_state, cost

### 1.2.4 | Apply Actions.

The function to apply a series of actions to an initial state is defined as "apply_actions(state,actions)".

This function calculates the total cost of the series of actions, as well as returning the state that is reached after applying them all sequentially.

In [ ]:
def apply_actions(state,actions):

    new_state = state
    total_cost = 0

    for action in actions:
        new_state, cost = apply_action(new_state, action)
        total_cost = total_cost + cost
        
    return new_state, total_cost

___

## 1.3 | Searching Algorithm.

### 1.3.1 | Heuristic.

This function calculates an estimate of the cost between a given state and the goal state, and is defined as "heuristic(state)".

It works by initially unpacking the state, ignoring held container as cost estimations are done based on the plan of what to pick up rather than the current held container.

It then estimates the cost of removing any obstacles from the target bay, by adding the cost to pick an obstacle, the cost to move it, and the cost to drop it

It then estimates the cost of getting to the next necessary container, moving it to the target bay, and placing it correctly.

All of this is returned as the total heuristic, providing a good estimate of the cost between a given state and the goal. This heuristic should not overestimate the cost and therefore be both optimal. If the tree is finite, it is also complete.

There is room for improvement, however this is a great trade off between function and readability.

In [ ]:
def heuristic(state):

    heuristic = 0
    
    bays, agent_pos, _, target_bay, weights = state

    for i in range(len(bays[target_bay])):
        if bays[target_bay][i] != i + 1:
            heuristic += 4 - len(bays[target_bay])
            heuristic += abs(agent_pos - target_bay) * (5 if weights.get(bays[target_bay][i]) == "heavy" else 3)
            heuristic += 2

    for i in range(1, 4):
        if i not in bays[target_bay]:
            for bay_pos, bay in enumerate(bays):
                if i in bay:
                    heuristic += abs(agent_pos - bay_pos) * (5 if weights.get(i) == "heavy" else 3)
                    heuristic += 4 - len(bay)
                    heuristic += abs(bay_pos - target_bay) * (5 if weights.get(i) == "heavy" else 3)
                    heuristic += 2
                    break

    return heuristic

### 1.3.2 | A* Search.

This A* search function works by importing heapq, which is a more efficient way of managing priority than manually managing the frontier.

It then initialises the frontier, and explored states.

The search takes the highest priority state, checks if it is the goal, adds it to explored, and then moves on to continue the search if these conditions aren't met.

The search tries each action by testing its validitiy, then applying it. This is returned as a cost, which is added to the heuristic of that state.

This is then returned as the new priority for the state, and added to the frontier.

It will loop through this until a solution is found.

In [ ]:
import heapq

def a_star_search(initial_state):
    
    frontier = [(heuristic(initial_state), initial_state, [], 0)]
    heapq.heapify(frontier)
    explored = set()

    while frontier:
        _, current_state, path, path_cost = heapq.heappop(frontier)

        if str(current_state) in explored:
            continue

        if is_goal_state(current_state):
            return path

        explored.add(str(current_state))

        for action in all_actions:
            if not is_action_valid(current_state, action, path[-1] if path else None):
                continue

            new_state, cost = apply_action(current_state, action)
            new_path = path + [action]
            new_path_cost = path_cost + cost
            priority = new_path_cost + heuristic(new_state)
            heapq.heappush(frontier, (priority, new_state, new_path, new_path_cost))

    return None

___

## 1.4 | Algorithm Results.

### 1.4.1 | Path Calculation.

This is where we call the search on the initial state. It will return the solution found, along with the total cost of the path.

It works generally in under 5 seconds on trees of a complexity of less than 50 actions, which is incredibly rare to exceed even when the puzzle is fully randomised.

The initial state provided should return a path of 37 actions.

In [ ]:
path = a_star_search(initial_state)

if path is None:
    print("No solution found.")
else:
    final_state, total_cost = apply_actions(initial_state, path)
    print_state(final_state)
    print("Path to goal:", path)
    print("\nNumber of actions:", len(path))
    print("\nTotal cost:", total_cost)

### 1.4.2 | Path Animation.

A small addition to animate the path using the print_state function, and clearing the output.

This path makes what appears to be a mistake, by not moving 3 up near the goal bay before moving 2, however the cost is identical. This was tested by grabbing the path output, and manually changing it to what seemed to make more sense.

In [ ]:
import time
from IPython.display import clear_output

def animate_path(initial_state, path):
        
        current_state = copy.deepcopy(initial_state)
        
        for action in path:
            current_state, _ = apply_action(current_state, action)
            clear_output(wait=True)
            print_state(current_state)
            time.sleep(0.2)

animate_path(initial_state, path)

final_state, total_cost = apply_actions(initial_state, path)

print("Path to goal:", path)
print("\nNumber of actions:", len(path))
print("\nTotal cost:", total_cost)

___

## 1.5 | Algorithm Performance on a Random Input.

**The entirety of section 1.5 can be re-run to test the algorithm on a new random input.**

### 1.5.1 | Containers Randomiser.

Considering the performance of the search on a relatively complex input, it can be tested on a completely random input.

This function allows the containers to be distributed randomly throughout a blank initial state.

In [ ]:
import random

def distribute_containers(bays):

    bays = copy.deepcopy(bays)
    containers = list(range(1, 7))
    
    random.shuffle(containers)

    for container in containers:
        random.shuffle(bays)

        for bay in bays:
            if len(bay) < 3:
                bay.append(container)
                break
    
    return bays

### 1.5.2 | Random Initial State.

This function builds the random intial state, by generating a random value for all variables, except held container.

In [ ]:
blank_bays = [[],[],[],[],[],[],[],[]]
random_target_bay = random.randint(0, 7)
random_agent_pos = random.randint(0, 7)
random_bays = distribute_containers(blank_bays)
random_state = [random_bays, random_agent_pos, initial_held_container, random_target_bay, weight]

print_state(random_state)

### 1.5.3 | Random Path Calculation.

This calls the search on the random initial state, and prints the solution.

On average, this is significantly less complex than the first initial state.

In [ ]:
random_state_path = a_star_search(random_state)
if path is None:
    print("No solution found.")
else:
    random_final_state, random_total_cost = apply_actions(random_state, random_state_path)

    print_state(random_final_state)
    print("Path to goal:", random_state_path)
    print("\nNumber of actions:", len(random_state_path))
    print("\nTotal cost:", random_total_cost)

### 1.5.4 | Random Path Animation.

This animates the path found for the random initial state.

In [ ]:
animate_path(random_state, random_state_path)

random_final_state, random_total_cost = apply_actions(random_state, random_state_path)

print("Path to goal:", random_state_path)
print("\nNumber of actions:", len(random_state_path))
print("\nTotal cost:", random_total_cost)

___
## **2 | Machine Learning Implementation.**

___
## 2.1 | Data Preparation

### 2.1.1 | Reading File.

This section reads the csv file containing the data to be trained and tested on.

In [ ]:
import pandas as pd
rawdata = pd.read_csv('ContainerData.csv')

### 2.1.2 | Target & Feature Selection.

Here the target and features are selected, informing the model what to predict for, and what to query the model with when testing.

In [ ]:
target = ["Priority"]
features = ["Height","Width","Hue","Times moved","Hours"]

### 2.1.3 | Spltting Testing and Training Data.

Here the training and test data are split, with a test size of 0.25, meaning 25% of the data will be omitted from training to use as test data.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(rawdata, test_size=0.25)

___
## 2.2 | Training.

### 2.2.1 | Training Variables.

Here we rename the split data into four variables, for readability.

train_target is flattened into a one dimesnional array to avoid an error when applying the model.

In [ ]:
import numpy as np

train_features = train[features]
train_target = train[target].to_numpy()

test_features = test[features]
test_target = test[target].to_numpy()

train_target = np.ravel(train_target)

### 2.2.2 | Selecting Machine Learning Model.

Classification is selected as the model, as it is most suited for predicting where the solution is one of a few specific options. 'High' and 'Low' priority classification are a perfect use case for this model.

In [ ]:
from sklearn.linear_model import LogisticRegression

Classification = LogisticRegression()

### 2.2.3 | Training the Model.

Here the model is given the training data, and adjusts appropriately.

In [ ]:
Classification.fit(train_features,train_target)

___
## 2.3 | Machine Learning Results.

### 2.3.1 | Generating Predicitons.

The testing data, with the exception of the target, is given to the model trained above.

In [ ]:
predictions_on_test_set = Classification.predict(test_features)

### 2.3.2 | Evaluating Accuracy.

This final section outputs the accuracy when comparing the expected classification with the predicted classification.

The output should be around 0.9, which is a 90% accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_target, predictions_on_test_set)

print("Accuracy: "+str(accuracy))